<a href="https://colab.research.google.com/github/inesclopes/data-eng-proc/blob/main/spark/examples/05-aggregations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/examples/05-aggregations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregations
- Group By
- Windows Functions

# Setting up PySpark

In [ ]:
%pip install pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').config('spark.ui.port', '4050').getOrCreate()

# Aggregations

https://spark.apache.org/docs/latest/sql-ref-functions-builtin.html#aggregate-functions

https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-aggregate.html

In [68]:
sql_query = """CREATE OR REPLACE TEMPORARY VIEW basic_pays AS SELECT * FROM VALUES
('Diane Murphy','Accounting',8435),
('Mary Patterson','Accounting',9998),
('Jeff Firrelli','Accounting',8992),
('William Patterson','Accounting',8870),
('Gerard Bondur','Accounting',11472),
('Anthony Bow','Accounting',6627),
('Leslie Jennings','IT',8113),
('Leslie Thompson','IT',5186),
('Julie Firrelli','Sales',9181),
('Steve Patterson','Sales',9441),
('Foon Yue Tseng','Sales',6660),
('George Vanauf','Sales',10563),
('Loui Bondur','SCM',10449),
('Gerard Hernandez','SCM',6949),
('Pamela Castillo','SCM',11303),
('Larry Bott','SCM',11798),
('Barry Jones','SCM',10586)
AS basic_pays(employee_name, department, salary)"""

# creating temp view
spark.sql(sql_query)

df = spark.table("basic_pays")
df.show()


+-----------------+----------+------+
|    employee_name|department|salary|
+-----------------+----------+------+
|     Diane Murphy|Accounting|  8435|
|   Mary Patterson|Accounting|  9998|
|    Jeff Firrelli|Accounting|  8992|
|William Patterson|Accounting|  8870|
|    Gerard Bondur|Accounting| 11472|
|      Anthony Bow|Accounting|  6627|
|  Leslie Jennings|        IT|  8113|
|  Leslie Thompson|        IT|  5186|
|   Julie Firrelli|     Sales|  9181|
|  Steve Patterson|     Sales|  9441|
|   Foon Yue Tseng|     Sales|  6660|
|    George Vanauf|     Sales| 10563|
|      Loui Bondur|       SCM| 10449|
| Gerard Hernandez|       SCM|  6949|
|  Pamela Castillo|       SCM| 11303|
|       Larry Bott|       SCM| 11798|
|      Barry Jones|       SCM| 10586|
+-----------------+----------+------+



In [69]:
(df.groupBy("department").agg(sum("salary")).show())

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|     Sales|      35845|
|Accounting|      54394|
|       SCM|      51085|
|        IT|      13299|
+----------+-----------+



In [6]:
perc_query = """SELECT
    department,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary) AS pc1,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary) FILTER (WHERE employee_name LIKE '%Bo%') AS pc2,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary DESC) AS pc3,
    percentile_cont(0.25) WITHIN GROUP (ORDER BY salary DESC) FILTER (WHERE employee_name LIKE '%Bo%') AS pc4,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary) AS pd1,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary) FILTER (WHERE employee_name LIKE '%Bo%') AS pd2,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary DESC) AS pd3,
    percentile_disc(0.25) WITHIN GROUP (ORDER BY salary DESC) FILTER (WHERE employee_name LIKE '%Bo%') AS pd4
FROM basic_pays
GROUP BY department
ORDER BY department;"""

spark.sql(perc_query).show()

+----------+-------+--------+-------+--------+-------+-------+-------+-------+
|department|    pc1|     pc2|    pc3|     pc4|    pd1|    pd2|    pd3|    pd4|
+----------+-------+--------+-------+--------+-------+-------+-------+-------+
|Accounting|8543.75| 7838.25| 9746.5|10260.75| 8435.0| 6627.0| 9998.0|11472.0|
|        IT|5917.75|    NULL|7381.25|    NULL| 5186.0|   NULL| 8113.0|   NULL|
|       SCM|10449.0|10786.25|11303.0|11460.75|10449.0|10449.0|11303.0|11798.0|
|     Sales|8550.75|    NULL| 9721.5|    NULL| 6660.0|   NULL|10563.0|   NULL|
+----------+-------+--------+-------+--------+-------+-------+-------+-------+



In [70]:
from pyspark.sql.functions import *
(df
 .groupBy("department")
 .agg(sum("salary").alias("sum_salary"),
      round(avg("salary"),2).alias("avg_salary"),
      min("salary").alias("min_salary"),
      array_agg("employee_name").alias("employees"),
      count(lit("")).alias("count_employees"))
 .filter(col("count_employees") > 2)
 .show(10, truncate=False))

+----------+----------+----------+----------+--------------------------------------------------------------------------------------------+---------------+
|department|sum_salary|avg_salary|min_salary|employees                                                                                   |count_employees|
+----------+----------+----------+----------+--------------------------------------------------------------------------------------------+---------------+
|Sales     |35845     |8961.25   |6660      |[Julie Firrelli, Steve Patterson, Foon Yue Tseng, George Vanauf]                            |4              |
|Accounting|54394     |9065.67   |6627      |[Diane Murphy, Mary Patterson, Jeff Firrelli, William Patterson, Gerard Bondur, Anthony Bow]|6              |
|SCM       |51085     |10217.0   |6949      |[Loui Bondur, Gerard Hernandez, Pamela Castillo, Larry Bott, Barry Jones]                   |5              |
+----------+----------+----------+----------+-------------------------

In [71]:
df.show()


+-----------------+----------+------+
|    employee_name|department|salary|
+-----------------+----------+------+
|     Diane Murphy|Accounting|  8435|
|   Mary Patterson|Accounting|  9998|
|    Jeff Firrelli|Accounting|  8992|
|William Patterson|Accounting|  8870|
|    Gerard Bondur|Accounting| 11472|
|      Anthony Bow|Accounting|  6627|
|  Leslie Jennings|        IT|  8113|
|  Leslie Thompson|        IT|  5186|
|   Julie Firrelli|     Sales|  9181|
|  Steve Patterson|     Sales|  9441|
|   Foon Yue Tseng|     Sales|  6660|
|    George Vanauf|     Sales| 10563|
|      Loui Bondur|       SCM| 10449|
| Gerard Hernandez|       SCM|  6949|
|  Pamela Castillo|       SCM| 11303|
|       Larry Bott|       SCM| 11798|
|      Barry Jones|       SCM| 10586|
+-----------------+----------+------+



In [72]:
df.withColumn("department", concat(col("department"), lit("_ts"))).show()

+-----------------+-------------+------+
|    employee_name|   department|salary|
+-----------------+-------------+------+
|     Diane Murphy|Accounting_ts|  8435|
|   Mary Patterson|Accounting_ts|  9998|
|    Jeff Firrelli|Accounting_ts|  8992|
|William Patterson|Accounting_ts|  8870|
|    Gerard Bondur|Accounting_ts| 11472|
|      Anthony Bow|Accounting_ts|  6627|
|  Leslie Jennings|        IT_ts|  8113|
|  Leslie Thompson|        IT_ts|  5186|
|   Julie Firrelli|     Sales_ts|  9181|
|  Steve Patterson|     Sales_ts|  9441|
|   Foon Yue Tseng|     Sales_ts|  6660|
|    George Vanauf|     Sales_ts| 10563|
|      Loui Bondur|       SCM_ts| 10449|
| Gerard Hernandez|       SCM_ts|  6949|
|  Pamela Castillo|       SCM_ts| 11303|
|       Larry Bott|       SCM_ts| 11798|
|      Barry Jones|       SCM_ts| 10586|
+-----------------+-------------+------+



In [90]:
from pyspark.sql import functions as F

# Question

In [ ]:
# Q1
# Aggregate data by surname
# Calculate highest salary by surname
# Include the respective employee that has the highest salary
# Include department information about this employee
# Count how many employees has that surname
# Put in an array all the first_names of the respective surname ordered


# schema expected:
# surname | count_employees | highest_salary | employee_with_highest_salary | department_with_highest_salary | array_with_all_the_first_names |

In [86]:
(df
 .withColumn("array", split('employee_name',"\s"))
 .withColumn("surname", last(col("array")))

 .show())

AnalysisException: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses.;
Aggregate [employee_name#1624, department#1625, salary#1626, array#2001, last(array#2001, false) AS surname#2007]
+- Project [employee_name#1624, department#1625, salary#1626, split(employee_name#1624, \s, -1) AS array#2001]
   +- SubqueryAlias basic_pays
      +- View (`basic_pays`, [employee_name#1624,department#1625,salary#1626])
         +- Project [cast(employee_name#1627 as string) AS employee_name#1624, cast(department#1628 as string) AS department#1625, cast(salary#1629 as int) AS salary#1626]
            +- Project [employee_name#1627, department#1628, salary#1629]
               +- SubqueryAlias basic_pays
                  +- LocalRelation [employee_name#1627, department#1628, salary#1629]


In [111]:
temp_agg_df = (df
 .withColumn("surname",
             F.split('employee_name',"\s")
 .getItem(1))
 .withColumn("fname",
             F.split('employee_name',"\s")
 .getItem(0))
 .groupBy("surname")
 .agg(max("salary")
  .alias("max_salary_by_surname"),
  F.count(lit(""))
  .alias("count_employees")
      )
 )

In [112]:
temp_agg_df.join(df, on =temp_agg_df['max_salary_by_surname']== df["salary"], how='left').show()

+---------+---------------------+---------------+----------------+----------+------+
|  surname|max_salary_by_surname|count_employees|   employee_name|department|salary|
+---------+---------------------+---------------+----------------+----------+------+
|      Bow|                 6627|              1|     Anthony Bow|Accounting|  6627|
|    Jones|                10586|              1|     Barry Jones|       SCM| 10586|
|   Bondur|                11472|              2|   Gerard Bondur|Accounting| 11472|
|   Murphy|                 8435|              1|    Diane Murphy|Accounting|  8435|
| Castillo|                11303|              1| Pamela Castillo|       SCM| 11303|
| Firrelli|                 9181|              2|  Julie Firrelli|     Sales|  9181|
|   Vanauf|                10563|              1|   George Vanauf|     Sales| 10563|
|      Yue|                 6660|              1|  Foon Yue Tseng|     Sales|  6660|
|Patterson|                 9998|              3|  Mary Patterson